# 线性回归的从零开始实现

In [ ]:
import random
import tensorflow as tf
from d2l import tensorflow as d2l

In [ ]:
def synthetic_data(w, b, num_examples):
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    